# Packed Column Model Simulation in IDAES for Post Combustion Capture

This example will demonstrate the use of IDAES Packed Column unit model for carbon capture applications. 
The Packed Column model can be used for absorption or stripping operation and is based on the enhancement factor approach where Monoethanolamine(MEA) is the solvent. For this example, the packed column is simulated in absorption mode. 

A brief overview of the Packed Column model is provided in this link:
https://idaes-pse--140.org.readthedocs.build/en/140/technical_specs/model_libraries/power_generation/carbon_capture/mea_solvent_system/unit_models/index.html

Reference Paper: Akula, Paul; Eslick, John; Bhattacharyya, Debangsu; Miller, David
"Model Development, Validation, and Part-Load Optimization of a
MEA-Based Post-Combustion CO2 Capture Process
Under Part-Load and Variable Capture Operation,
Industrial & Engineering Chemistry Research,2021. (submitted)

### Import Pyomo libraries

In [ ]:
import pyomo.environ as pyo

### Import the IDAES Model Libraries

In [ ]:
# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
import idaes
from idaes.core import FlowsheetBlock

# Liquid and Vapor property package
from idaes.power_generation.carbon_capture.mea_solvent_system.properties.liquid_prop import LiquidParameterBlock
from idaes.power_generation.carbon_capture.mea_solvent_system.properties.vapor_prop import VaporParameterBlock

# Packed column unit model
from idaes.power_generation.carbon_capture.mea_solvent_system.unit_models.column import PackedColumn, ProcessType

# Solver
from idaes.core.util import get_solver

### Create a Pyomo concrete model and attach the IDAES flowsheet block to it


In [ ]:
m = pyo.ConcreteModel()
m.fs = FlowsheetBlock(default={"dynamic": False})

### Set up the property package for vapor and liquid phase of the column

In [ ]:
m.fs.vapor_properties = VaporParameterBlock(default={'process_type': ProcessType.absorber})
m.fs.liquid_properties = LiquidParameterBlock(default={'process_type': ProcessType.absorber})

### Set up the packed column unit model

In [ ]:
# Spatial domain of finite elements and finite element list
x_nfe = 40
x_nfe_list = [i / x_nfe for i in range(x_nfe + 1)]

# Create an instance of the packed column in the flowsheet for absorption process (provide specifications)
m.fs.abs = PackedColumn(default={
            "process_type": ProcessType.absorber,
            "finite_elements": x_nfe,
            "length_domain_set": x_nfe_list,
            "column_pressure": 107650, # Pa
            "packing_specific_area":225, # m^2/m^3
            "packing_void_fraction":0.97,
            "transformation_method": "dae.finite_difference",
            "vapor_side": {
                "transformation_scheme": "BACKWARD",
                "property_package": m.fs.vapor_properties,
                "has_pressure_change": False,
                "pressure_drop_type": None},
            "liquid_side":
            {
                "transformation_scheme": "FORWARD",
                "property_package": m.fs.liquid_properties
            }})

 ### Initialize and solve the  absorber model
 
 1. Fix dimensions of absorber (according to TCM data)
 2. Fix the process inputs using data from Faramarzia et. al, 2017.
 3. Initialize the model
 4. Solve the model(scale up to required diameter)

In [ ]:
# Absorber diameter and length
m.fs.abs.diameter_column.fix(0.6) # m
m.fs.abs.length_column.fix(24) # m

# Vapor inputs (flue gas)
m.fs.abs.vapor_inlet.flow_mol[0].fix(698.218) # mol/s
m.fs.abs.vapor_inlet.temperature[0].fix(302.95) # K
m.fs.abs.vapor_inlet.pressure[0].fix(107625.0) # Pa
m.fs.abs.vapor_inlet.mole_frac_comp[0, "CO2"].fix(0.037)
m.fs.abs.vapor_inlet.mole_frac_comp[0, "H2O"].fix(0.037)
m.fs.abs.vapor_inlet.mole_frac_comp[0, "N2"].fix(0.78)
m.fs.abs.vapor_inlet.mole_frac_comp[0, "O2"].fix(0.146)

# Liquid inputs (solvent)
m.fs.abs.liquid_inlet.flow_mol[0].fix(677.774) # mol/s
m.fs.abs.liquid_inlet.temperature[0].fix(310.15) # K
m.fs.abs.liquid_inlet.mole_frac_comp[0, "CO2"].fix(0.0229)
m.fs.abs.liquid_inlet.mole_frac_comp[0, "H2O"].fix(0.8628)
m.fs.abs.liquid_inlet.mole_frac_comp[0, "MEA"].fix(0.1143)

# Access the solver
solver = get_solver()

# Initialization 
m.fs.abs.initialize(homotopy_steps_h=[0.8, 0.9, 1])
import numpy as np
abs_col_diameter = np.linspace(0.6, 3.007, num=10)
for D in abs_col_diameter:
  m.fs.abs.diameter_column.fix(D)
  print('column diameter = {:6.2f}'.format(D))
  solver.solve(m.fs.abs,tee=False)

### View model results

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

m.fs.abs.make_steady_state_column_profile()

### Model validation with TCM pilot plant data

In [ ]:
# Get the TCM data
TCM_abs_Temp_profile= [305.55,305.65,306.35,306.05,306.95,307.25,307.45,309.05,310.25,310.55,314.75,313.75,314.05,
                       315.35, 316.25, 317.55, 319.15,  320.35,  322.05,323.05,323.65,324.75,324.85,320.55,310.15]

TCM_abs_height=[0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5, 8.5, 9.5, 10.5, 11.5, 12.5,
                13.5, 14.5, 15.5,16.5, 17.5, 18.5, 19.5, 20.5, 21.5, 22.5, 23.5, 24]

# Get model result
abs_column_height = [x*24 for x in m.fs.abs.liquid_phase.length_domain]

liquid_temperature_profile = []
for x in m.fs.abs.liquid_phase.length_domain:
    liquid_temperature_profile.append(pyo.value(m.fs.abs.liquid_phase.properties[0, x].temperature))

# Validation plot
plt.figure(figsize=(11,9))

plt.plot(TCM_abs_height, TCM_abs_Temp_profile,
         label='TCM data : 2015',color ='black',
         marker='o',markersize=9,linestyle='')


plt.plot(abs_column_height,liquid_temperature_profile,
         color='blue',label='Model prediction')


plt.ylim(295,335)
plt.ylabel('Temperature (K)',fontsize=19,fontweight='bold')
plt.xlabel('Absorber height from bottom',fontsize=19,fontweight='bold')
plt.legend(fontsize=19)
plt.tick_params(labelsize=19)
plt.tight_layout()